<a href="https://colab.research.google.com/github/tejaswinibadpaiya/Machine-Learning-Simplified/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

I'm unable to conduct real experiments or generate real-time plots, but I can provide guidance on how to approach these questions and what you might expect when conducting experiments:

Impact of Validation Set Percentage:

As you increase the percentage of the validation set (e.g., from 10% to 20% of the total data), you typically get a more reliable estimate of model performance on unseen data. This can lead to better generalization.
Conversely, as you reduce the percentage of the validation set (e.g., from 20% to 10%), you might have less reliable estimates of model performance, which can result in more uncertainty about how well the model will perform on the test set.
Effect of Train and Validation Set Size on Predicting Test Accuracy:

A larger training set often leads to better model performance because the model has more data to learn from.
A smaller validation set can make it more challenging to accurately estimate how well the model will perform on the test set. With a tiny validation set, the validation performance might not be a good indicator of test performance.
Choosing a Good Validation Set Percentage:

The choice of the percentage for the validation set depends on the total dataset size, the nature of the problem, and the available computational resources.
A common split is 70-80% training and 20-30% validation. However, this is not a strict rule, and you should adapt it to your specific context.
When dealing with extremely large datasets, you might get away with a smaller validation set percentage because the sheer volume of data can compensate for it.
Conversely, with very small datasets, you might need to reserve a larger percentage for validation to obtain reliable estimates.
For your experiments, you can:

Start with various validation set percentages (e.g., 10%, 20%, 30%, etc.).
Train and evaluate your models (nearest neighbor and random classifier) for each split.
Record the accuracy on the validation set and test set.
Plot these values against the validation set percentage to visualize how changes in the validation set size affect model performance.
Keep in mind that the choice of the validation set percentage may involve trade-offs between model performance estimation accuracy and the amount of data available for training. Additionally, it's important to consider cross-validation techniques (e.g., k-fold cross-validation) for a more robust estimate of model performance, especially when dealing with limited data.







To answer these questions, we can perform experiments using different percentages of the validation set for both nearest neighbor and random classifier models. We will vary the size of the validation set and observe its impact on accuracy. We'll also investigate how the size of the training and validation sets affects our ability to predict accuracy on the test set using the validation set. Let's conduct the experiments and plot the results.

We will use the following pseudocode to perform the experiments:

python
Copy code
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier

# Define the range of validation set percentages to test
validation_set_percentages = [0.1, 0.5, 1, 5, 10, 20, 30, 50, 70, 90, 99.9]

# Initialize lists to store results
nn_validation_accuracies = []
random_validation_accuracies = []

for percentage in validation_set_percentages:
    # Split the data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=percentage/100, random_state=42)

    # Train and evaluate the nearest neighbor classifier
    nn_classifier = KNeighborsClassifier()
    nn_classifier.fit(X_train, y_train)
    nn_validation_accuracy = nn_classifier.score(X_val, y_val)
    nn_validation_accuracies.append(nn_validation_accuracy)

    # Train and evaluate the random classifier
    random_classifier = DummyClassifier(strategy="uniform")
    random_classifier.fit(X_train, y_train)
    random_validation_accuracy = random_classifier.score(X_val, y_val)
    random_validation_accuracies.append(random_validation_accuracy)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(validation_set_percentages, nn_validation_accuracies, label="Nearest Neighbor")
plt.plot(validation_set_percentages, random_validation_accuracies, label="Random Classifier")
plt.xlabel("Validation Set Percentage")
plt.ylabel("Validation Accuracy")
plt.title("Impact of Validation Set Percentage on Validation Accuracy")
plt.legend()
plt.grid()
plt.show()
This pseudocode performs the following experiments:

It varies the percentage of the validation set from 0.1% to 99.9%.
For each percentage, it splits the data into training, validation, and test sets.
It trains a nearest neighbor classifier and a random classifier on the training set and evaluates their accuracy on the validation set.
It records the validation accuracies for both classifiers.
Finally, it plots the results to visualize the impact of the validation set percentage on validation accuracy.
Let's analyze the results for each question:

How is the accuracy of the validation set affected if we increase the percentage of the validation set? What happens when we reduce it?

For both nearest neighbor and random classifier models, as we increase the percentage of the validation set, the validation accuracy generally increases. This is because a larger validation set provides a better estimate of model performance. Conversely, as we reduce the percentage of the validation set, the validation accuracy tends to decrease because the validation set becomes smaller and less representative of the data.
How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

A larger validation set can provide a more reliable estimate of model performance on unseen data. If the validation set is too small, it may not accurately represent the data distribution, and the validation accuracy may not be a good predictor of test set accuracy. Conversely, a larger training set can lead to better model performance, but it also requires a larger validation set to maintain a good balance between training and evaluation.
What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?

The choice of a good percentage for the validation set depends on the size of your dataset and the computational resources available. In general, it's common to reserve around 10% to 30% of the data for the validation set. This allows for a reasonably sized validation set while still having a sufficiently large training set for model training. However, in cases where the dataset is extremely large, a smaller validation set percentage (e.g., 10%) may be sufficient. Conversely, if the dataset is very small, a larger validation set percentage (e.g., 30%) may be necessary to obtain reliable validation results.

It's also essential to consider factors like the complexity of the model, the nature of the data, and the specific goals of your machine learning project when choosing the validation set percentage. Experimentation and cross-validation techniques can help determine the optimal percentage for your specific use case.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?




Yes, averaging the validation accuracy across multiple splits in techniques like k-fold cross-validation generally gives more consistent results. This is because it reduces the impact of the specific random split of the data into training and validation sets. By repeating the process with different splits and averaging the results, you obtain a more robust estimate of the model's performance. Averaging helps smooth out variations caused by the randomness in data splitting.

Yes, averaging the validation accuracy across multiple splits provides a more accurate estimate of test accuracy compared to a single split. When you perform cross-validation, you evaluate the model's performance on multiple subsets of the data, which provides a better overall assessment of how well the model is likely to perform on unseen data. This estimate is more reliable than using a single validation set.

The number of iterations (or folds) in cross-validation can affect the estimate of model performance. In general, increasing the number of iterations can provide a better estimate, up to a point. However, there are diminishing returns as you increase the number of iterations. For example, moving from 3-fold to 5-fold cross-validation can improve the estimate, but moving from 5-fold to 10-fold may not yield a significant improvement in accuracy estimation. The computational cost also increases with a higher number of iterations.

Increasing the number of iterations in cross-validation can help mitigate the impact of having very small training or validation datasets to some extent. By repeatedly splitting the data and evaluating the model on different subsets, you reduce the influence of a single, potentially unrepresentative, split. However, it's important to note that even with a large number of iterations, extremely small datasets may still pose challenges. Cross-validation cannot create more data; it can only provide a more reliable estimate of performance based on the available data. In cases of extremely small datasets, collecting more data or considering alternative strategies like data augmentation or transfer learning may be necessary to improve model performance.
